# Note

Before running all, if you so chose, there are cells that ask for input from the user. So, if you don't want to provide input, scroll down and run the cells with the manual approach. If you want to recreate what I showed in the video, go the the Manual Approach cell and change the file name in the first function accordingly.

In [1]:
from scipy.optimize import linprog
import pandas as pd
import numpy as np

### Helper Functions

These are just the functions used. Scroll down to test.

In [2]:
def read_data(file):
    data = pd.read_csv(file, skiprows=[0,2], header=1)
    data.drop(columns=['Last Sale', 'Last Sale.1', 'Net', 'Net.1', 'IV', 'IV.1', 'Delta', 'Delta.1', 
                   'Gamma', 'Gamma.1', 'Volume', 'Volume.1', 'Open Interest', 'Open Interest.1'], inplace=True)
    data.columns = ['Expiration Date', 'Call ID', 'Call Bid', 'Call Ask', 'Strike', 'Put ID', 'Put Bid', 'Put Ask']
    data['Expiration Date'] = pd.to_datetime(data['Expiration Date'])
    return data

def selectFile(file=None):
    if file is not None:
        try:
            data = read_data(file)
            print('You selected:', file, '\n')
            return data
        except:
            print('File not found. Please try again.\n')
    while True:
        file = input('Enter file name (i.e. options.csv): ')
        try:
            data = read_data(file)
            print('You selected:', file, '\n')
            return data
        except:
            print('File not found. Please try again.\n')

def choosePeriod(data, period=None):
    if period is not None:
        if period.lower() == 'weekly':
            weekly_data = data[data['Call ID'].str.contains('W')].reset_index(drop=True)
            if len(weekly_data) == 0:
                print('Only monthly options available.\n')
                return None
            print('You selected:', period, '\n')
            return weekly_data
        elif period.lower() == 'monthly':
            monthly_data = data[~data['Call ID'].str.contains('W')].reset_index(drop=True)
            if len(monthly_data) == 0:
                print('Only weekly options available.\n')
                return None
            print('You selected:', period, '\n')
            return monthly_data
        else:
            print('Please enter weekly or monthly.\n')
    weekly_data = data[data['Call ID'].str.contains('W')].reset_index(drop=True)
    monthly_data = data[~data['Call ID'].str.contains('W')].reset_index(drop=True)
    if len(weekly_data) > 0 and len(monthly_data) > 0:
        while True:
            period = input('Enter period (weekly/monthly): ')
            if period.lower() == 'weekly':
                print('You selected:', period, '\n')
                return weekly_data
            elif period.lower() == 'monthly':
                print('You selected:', period, '\n')
                return monthly_data
            else:
                print('You selected:', period)
                print('Please enter weekly or monthly.\n')
    elif len(weekly_data) > 0:
        print('Only weekly options available.\n')
        return weekly_data
    else:
        print('Only monthly options available.\n')
        return monthly_data
    
def choseExpiration(data, expiration=None):
    if expiration is not None:
        if expiration in data['Expiration Date'].unique():
            print('You selected:', expiration, '\n')
            return data[data['Expiration Date'] == expiration]
        else:
            print('You selected:', expiration)
            print('Please enter a valid expiration date.\n')
            return None
    expirations = data['Expiration Date'].unique()
    if len(expirations) == 1:
        print('Only one expiration available.\n')
        return data[data['Expiration Date'] == expirations[0]]
    while True:
        expiration = input('Enter expiration date (i.e. YYYY-MM-DD): ')
        if expiration in expirations:
            print('You selected:', expiration, '\n')
            return data[data['Expiration Date'] == expiration]
        else:
            print('You selected:', expiration)
            print('Please enter a valid expiration date.\n')

def createStrikes(data):
    if 0 in data['Strike'].values:
        return data['Strike'].values
    else:
        return np.insert(data['Strike'].values, 0, 0)
    
def createPriceVector(data):
    return np.concatenate((data['Call Ask'].values, -data['Call Bid'].values, data['Put Ask'].values, -data['Put Bid'].values))
    
def createCallMatrix(strikes):
    A = np.zeros((len(strikes) + 1, len(strikes) - 1))
    A[-1,:] = 1
    for i in range(1, len(strikes)):
        diff = strikes - strikes[i]
        diff[diff < 0] = 0
        A[:-1, i - 1] = diff
    return A

def createPutMatrix(strikes):
    A = np.zeros((len(strikes) + 1, len(strikes) - 1))
    for i in range(len(strikes) - 1, 0, -1):
        diff = strikes[i] - strikes
        diff[diff < 0] = 0
        A[:-1, i - 1] = diff
    return A

def createStrikeMatrix(strikes):
    A_call = createCallMatrix(strikes)
    A_put = createPutMatrix(strikes)
    A = np.hstack((A_call, -A_call, A_put, -A_put))
    return A

def explainArbitrage(data, res):
    count = data.shape[0]
    actions = ['Long Call', 'Short Call', 'Long Put', 'Short Put']
    for i in range(0, count * 4, count):
        positions = np.argwhere(res.x[i:i+count] > 1e-10)
        proportions = res.x[i:i+count][positions]
        for pos, pro in zip(positions, proportions):
            if i < count * 2:
                print(actions[int(i / count)], data['Call ID'].values[pos[0]], 'at strike', data['Strike'].values[pos[0]], 'with proportion:', round(pro[0], 3))
            else:
                print(actions[int(i / count)], data['Put ID'].values[pos[0]], 'at strike', data['Strike'].values[pos[0]], 'with proportion:', round(pro[0], 3))

    
def detectArbitrage():
    data = selectFile()
    data = choosePeriod(data)
    data = choseExpiration(data)
    strikes = createStrikes(data)
    A = createStrikeMatrix(strikes)
    c = createPriceVector(data)
    b = np.zeros(A.shape[0])
    res = linprog(c, A_ub=-A, b_ub=b, bounds=(0, None))
    if res.success:
        print('No arbitrage detected.\n')
    else:
        res = linprog(c, A_ub=-A, b_ub=b, A_eq=np.ones((1, A.shape[1])), b_eq=1, bounds=(0, None))
        print('Arbitrage detected.\n')
        explainArbitrage(data, res)
    return res

# Arbitrage Detection

I created two ways to test the LP. The first way is to run the detectArbitrage() function and input the requested information. The second way is to manually input the requested information into the functions selectFile(), choosePeriod() and chooseExpiration(). Also, the output of linprog is saved as the variable res for further inspection.

Additionally, I used the detectArbitrage() to test the 'SPX Options ver x.csv' files and output the results. I also used the manual input approach to loop through all of the possible options for the three provided full options chain files. The output of these tests can be seen below the two approaches that you can use to test the LP.

### detectArbitrage() Approach

In [13]:
res = detectArbitrage()

You selected: SPX Options ver 1.csv 

Only weekly options available.

Only one expiration available.

No arbitrage detected.



### Manual Approach

In [3]:
data = selectFile(file='SPX Options ver 1.csv')
data = choosePeriod(data, period='weekly')
test_data = choseExpiration(data, expiration='2021-12-17')
strikes = createStrikes(test_data)
A = createStrikeMatrix(strikes)
c = createPriceVector(test_data)
b = np.zeros(A.shape[0])
res = linprog(c, A_ub=-A, b_ub=b, bounds=(0, None))
if res.success:
    print('No arbitrage detected.\n')
else:
    res = linprog(c, A_ub=-A, b_ub=b, A_eq=np.ones((1, A.shape[1])), b_eq=1, bounds=(0, None))
    print('Arbitrage detected.\n')
    explainArbitrage(data, res)

You selected: SPX Options ver 1.csv 

You selected: weekly 

You selected: 2021-12-17 

No arbitrage detected.



# Testing 'SPX Options ver x.csv' Files

In [8]:
res = detectArbitrage()

You selected: SPX Options ver 1.csv 

Only weekly options available.

Only one expiration available.

No arbitrage detected.



In [9]:
res = detectArbitrage()

You selected: SPX Options ver 2.csv 

Only weekly options available.

Only one expiration available.

Arbitrage detected.

Long Call SPXW211217C03000000 at strike 3000 with proportion: 0.5
Short Call SPXW211217C03000000 at strike 3000 with proportion: 0.5


In [10]:
res = detectArbitrage()

You selected: SPX Options ver 3.csv 

Only weekly options available.

Only one expiration available.

Arbitrage detected.

Long Call SPXW211217C03000000 at strike 3000 with proportion: 0.429
Short Call SPXW211217C01000000 at strike 1000 with proportion: 0.143
Short Call SPXW211217C04000000 at strike 4000 with proportion: 0.286
Long Put SPXW211217P05000000 at strike 5000 with proportion: 0.143


In [9]:
res = detectArbitrage()

You selected: SPX Options ver 4.csv 

Only weekly options available.

Only one expiration available.

Arbitrage detected.

Long Call SPXW211217C04000000 at strike 4000 with proportion: 0.312
Short Call SPXW211217C01000000 at strike 1000 with proportion: 0.125
Long Put SPXW211217P02000000 at strike 2000 with proportion: 0.063
Long Put SPXW211217P06000000 at strike 6000 with proportion: 0.187
Short Put SPXW211217P04000000 at strike 4000 with proportion: 0.312


# Testing All Possible Options of Three Provided Files

## 'spx 24Aug2022.csv'

### Weekly

In [8]:
data = selectFile(file='spx 24Aug2022.csv')
data = choosePeriod(data, period='weekly')
for exp in data['Expiration Date'].unique():
    test_data = choseExpiration(data, expiration=exp.strftime('%Y-%m-%d'))
    strikes = createStrikes(test_data)
    A = createStrikeMatrix(strikes)
    c = createPriceVector(test_data)
    b = np.zeros(A.shape[0])
    res = linprog(c, A_ub=-A, b_ub=b, bounds=(0, None))
    if res.success:
        print('No arbitrage detected.\n')
    else:
        res = linprog(c, A_ub=-A, b_ub=b, A_eq=np.ones((1, A.shape[1])), b_eq=1, bounds=(0, None))
        print('Arbitrage detected.\n')
        explainArbitrage(data, res)

You selected: spx 24Aug2022.csv 

You selected: weekly 

You selected: 2022-08-24 

No arbitrage detected.

You selected: 2022-08-25 

No arbitrage detected.

You selected: 2022-08-26 

No arbitrage detected.

You selected: 2022-08-29 

No arbitrage detected.

You selected: 2022-08-30 

No arbitrage detected.

You selected: 2022-08-31 

No arbitrage detected.

You selected: 2022-09-01 

No arbitrage detected.

You selected: 2022-09-02 

No arbitrage detected.

You selected: 2022-09-06 

No arbitrage detected.

You selected: 2022-09-07 

No arbitrage detected.

You selected: 2022-09-08 

No arbitrage detected.

You selected: 2022-09-09 

No arbitrage detected.

You selected: 2022-09-12 

No arbitrage detected.

You selected: 2022-09-14 

No arbitrage detected.

You selected: 2022-09-16 

No arbitrage detected.

You selected: 2022-09-19 

No arbitrage detected.

You selected: 2022-09-21 

No arbitrage detected.

You selected: 2022-09-23 

No arbitrage detected.

You selected: 2022-09-30 

### Monthly

In [20]:
data = selectFile(file='spx 24Aug2022.csv')
data = choosePeriod(data, period='monthly')
for exp in data['Expiration Date'].unique():
    test_data = choseExpiration(data, expiration=exp.strftime('%Y-%m-%d'))
    strikes = createStrikes(test_data)
    A = createStrikeMatrix(strikes)
    c = createPriceVector(test_data)
    b = np.zeros(A.shape[0])
    res = linprog(c, A_ub=-A, b_ub=b, bounds=(0, None))
    if res.success:
        print('No arbitrage detected.\n')
    else:
        res = linprog(c, A_ub=-A, b_ub=b, A_eq=np.ones((1, A.shape[1])), b_eq=1, bounds=(0, None))
        print('Arbitrage detected.\n')
        explainArbitrage(data, res)

You selected: spx 24Aug2022.csv 

You selected: monthly 

You selected: 2022-09-16 

No arbitrage detected.

You selected: 2022-10-21 

No arbitrage detected.

You selected: 2022-11-18 

No arbitrage detected.

You selected: 2022-12-16 

No arbitrage detected.

You selected: 2023-01-20 

No arbitrage detected.

You selected: 2023-02-17 

No arbitrage detected.

You selected: 2023-03-17 

No arbitrage detected.

You selected: 2023-04-21 

No arbitrage detected.

You selected: 2023-05-19 

No arbitrage detected.

You selected: 2023-06-16 

No arbitrage detected.

You selected: 2023-07-21 

No arbitrage detected.

You selected: 2023-08-18 

No arbitrage detected.

You selected: 2023-09-15 

No arbitrage detected.

You selected: 2023-12-15 

No arbitrage detected.

You selected: 2024-06-21 

No arbitrage detected.

You selected: 2024-12-20 

No arbitrage detected.

You selected: 2025-12-19 

No arbitrage detected.

You selected: 2026-12-18 

No arbitrage detected.

You selected: 2027-12-17

## 'spx5Oct2022.csv'

### Weekly

In [3]:
data = selectFile(file='spx5Oct2022.csv')
data = choosePeriod(data, period='weekly')
for exp in data['Expiration Date'].unique():
    test_data = choseExpiration(data, expiration=exp.strftime('%Y-%m-%d'))
    strikes = createStrikes(test_data)
    A = createStrikeMatrix(strikes)
    c = createPriceVector(test_data)
    b = np.zeros(A.shape[0])
    res = linprog(c, A_ub=-A, b_ub=b, bounds=(0, None))
    if res.success:
        print('No arbitrage detected.\n')
    else:
        res = linprog(c, A_ub=-A, b_ub=b, A_eq=np.ones((1, A.shape[1])), b_eq=1, bounds=(0, None))
        print('Arbitrage detected.\n')
        explainArbitrage(data, res)

You selected: spx5Oct2022.csv 

You selected: weekly 

You selected: 2022-10-05 

No arbitrage detected.

You selected: 2022-10-06 

No arbitrage detected.

You selected: 2022-10-07 

No arbitrage detected.

You selected: 2022-10-10 

No arbitrage detected.

You selected: 2022-10-11 

No arbitrage detected.

You selected: 2022-10-12 

No arbitrage detected.

You selected: 2022-10-13 

No arbitrage detected.

You selected: 2022-10-14 

No arbitrage detected.

You selected: 2022-10-17 

No arbitrage detected.

You selected: 2022-10-18 

No arbitrage detected.

You selected: 2022-10-19 

No arbitrage detected.

You selected: 2022-10-20 

No arbitrage detected.

You selected: 2022-10-21 

No arbitrage detected.

You selected: 2022-10-24 

No arbitrage detected.

You selected: 2022-10-25 

No arbitrage detected.

You selected: 2022-10-26 

No arbitrage detected.

You selected: 2022-10-27 

No arbitrage detected.

You selected: 2022-10-28 

No arbitrage detected.

You selected: 2022-10-31 



### Monthly

In [4]:
data = selectFile(file='spx5Oct2022.csv')
data = choosePeriod(data, period='monthly')
for exp in data['Expiration Date'].unique():
    test_data = choseExpiration(data, expiration=exp.strftime('%Y-%m-%d'))
    strikes = createStrikes(test_data)
    A = createStrikeMatrix(strikes)
    c = createPriceVector(test_data)
    b = np.zeros(A.shape[0])
    res = linprog(c, A_ub=-A, b_ub=b, bounds=(0, None))
    if res.success:
        print('No arbitrage detected.\n')
    else:
        res = linprog(c, A_ub=-A, b_ub=b, A_eq=np.ones((1, A.shape[1])), b_eq=1, bounds=(0, None))
        print('Arbitrage detected.\n')
        explainArbitrage(data, res)

You selected: spx5Oct2022.csv 

You selected: monthly 

You selected: 2022-10-21 

No arbitrage detected.

You selected: 2022-11-18 

No arbitrage detected.

You selected: 2022-12-16 

No arbitrage detected.

You selected: 2023-01-20 

No arbitrage detected.

You selected: 2023-02-17 

No arbitrage detected.

You selected: 2023-03-17 

No arbitrage detected.

You selected: 2023-04-21 

No arbitrage detected.

You selected: 2023-05-19 

No arbitrage detected.

You selected: 2023-06-16 

No arbitrage detected.

You selected: 2023-07-21 

No arbitrage detected.

You selected: 2023-08-18 

No arbitrage detected.

You selected: 2023-09-15 

No arbitrage detected.

You selected: 2023-10-20 

No arbitrage detected.

You selected: 2023-12-15 

No arbitrage detected.

You selected: 2024-01-19 

No arbitrage detected.

You selected: 2024-06-21 

No arbitrage detected.

You selected: 2024-12-20 

No arbitrage detected.

You selected: 2025-12-19 

No arbitrage detected.

You selected: 2026-12-18 


## 'spx_27Dec23.csv'

### Weekly

In [5]:
data = selectFile(file='spx_27Dec23.csv')
data = choosePeriod(data, period='weekly')
for exp in data['Expiration Date'].unique():
    test_data = choseExpiration(data, expiration=exp.strftime('%Y-%m-%d'))
    strikes = createStrikes(test_data)
    A = createStrikeMatrix(strikes)
    c = createPriceVector(test_data)
    b = np.zeros(A.shape[0])
    res = linprog(c, A_ub=-A, b_ub=b, bounds=(0, None))
    if res.success:
        print('No arbitrage detected.\n')
    else:
        res = linprog(c, A_ub=-A, b_ub=b, A_eq=np.ones((1, A.shape[1])), b_eq=1, bounds=(0, None))
        print('Arbitrage detected.\n')
        explainArbitrage(data, res)

You selected: spx_27Dec23.csv 

You selected: weekly 

You selected: 2023-12-27 

No arbitrage detected.

You selected: 2023-12-28 

No arbitrage detected.

You selected: 2023-12-29 

No arbitrage detected.

You selected: 2024-01-02 

No arbitrage detected.

You selected: 2024-01-03 

No arbitrage detected.

You selected: 2024-01-04 

No arbitrage detected.

You selected: 2024-01-05 

No arbitrage detected.

You selected: 2024-01-08 

No arbitrage detected.

You selected: 2024-01-09 

No arbitrage detected.

You selected: 2024-01-10 

No arbitrage detected.

You selected: 2024-01-11 

No arbitrage detected.

You selected: 2024-01-12 

No arbitrage detected.

You selected: 2024-01-16 

No arbitrage detected.

You selected: 2024-01-17 

No arbitrage detected.

You selected: 2024-01-18 

No arbitrage detected.

You selected: 2024-01-19 

No arbitrage detected.

You selected: 2024-01-22 

No arbitrage detected.

You selected: 2024-01-23 

No arbitrage detected.

You selected: 2024-01-24 



### Monthly

In [6]:
data = selectFile(file='spx_27Dec23.csv')
data = choosePeriod(data, period='monthly')
for exp in data['Expiration Date'].unique():
    test_data = choseExpiration(data, expiration=exp.strftime('%Y-%m-%d'))
    strikes = createStrikes(test_data)
    A = createStrikeMatrix(strikes)
    c = createPriceVector(test_data)
    b = np.zeros(A.shape[0])
    res = linprog(c, A_ub=-A, b_ub=b, bounds=(0, None))
    if res.success:
        print('No arbitrage detected.\n')
    else:
        res = linprog(c, A_ub=-A, b_ub=b, A_eq=np.ones((1, A.shape[1])), b_eq=1, bounds=(0, None))
        print('Arbitrage detected.\n')
        explainArbitrage(data, res)

You selected: spx_27Dec23.csv 

You selected: monthly 

You selected: 2024-01-19 

No arbitrage detected.

You selected: 2024-02-16 

No arbitrage detected.

You selected: 2024-03-15 

No arbitrage detected.

You selected: 2024-04-19 

No arbitrage detected.

You selected: 2024-05-17 

No arbitrage detected.

You selected: 2024-06-21 

No arbitrage detected.

You selected: 2024-07-19 

No arbitrage detected.

You selected: 2024-08-16 

No arbitrage detected.

You selected: 2024-09-20 

No arbitrage detected.

You selected: 2024-10-18 

No arbitrage detected.

You selected: 2024-11-15 

No arbitrage detected.

You selected: 2024-12-20 

No arbitrage detected.

You selected: 2025-01-17 

No arbitrage detected.

You selected: 2025-03-21 

No arbitrage detected.

You selected: 2025-06-20 

No arbitrage detected.

You selected: 2025-12-19 

No arbitrage detected.

You selected: 2026-12-18 

No arbitrage detected.

You selected: 2027-12-17 

No arbitrage detected.

You selected: 2028-12-15 
